In [1]:
!pip install -q torch sentence-transformers scikit-learn
!pip install tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have rmm-cu12 25.2.0 which is incompatible.


In [2]:
import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


2025-04-30 01:31:25.146797: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745976685.400743      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745976685.469616      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:

path = '/kaggle/input/book-summary/books_summary.csv'
df = pd.read_csv(path)

print("Columns:", df.columns.tolist())
df.head(5)


Columns: ['Unnamed: 0', 'book_name', 'summaries', 'categories']


,Unnamed: 0,book_name,summaries,categories
0,0,The Highly Sensitive Person,is a self-assessment guide and how-to-live te...,science
1,1,Why Has Nobody Told Me This Before?,is a collection of a clinical psychologist’s ...,science
2,2,The Midnight Library,"tells the story of Nora, a depressed woman in...",science
3,3,Brave New World,presents a futuristic society engineered perf...,science
4,4,1984,is the story of a man questioning the system ...,science


In [4]:

subset = (
    df[['book_name','summaries','categories']]
    .dropna()
    .head(500)
    .reset_index(drop=True)
)

subset['text'] = subset['book_name'].str.strip() + '. ' + subset['summaries'].str.strip()

print("Prepared subset shape:", subset.shape)
subset[['book_name','categories','text']].head(5)


Prepared subset shape: (500, 4)


,book_name,categories,text
0,The Highly Sensitive Person,science,The Highly Sensitive Person. is a self-assessm...
1,Why Has Nobody Told Me This Before?,science,Why Has Nobody Told Me This Before?. is a coll...
2,The Midnight Library,science,"The Midnight Library. tells the story of Nora,..."
3,Brave New World,science,Brave New World. presents a futuristic society...
4,1984,science,1984. is the story of a man questioning the sy...


In [5]:
model_names = [
    'all-MiniLM-L6-v2',
    'paraphrase-MiniLM-L6-v2',
    'all-mpnet-base-v2'
]

models = {}
embeddings_dict = {}

for name in model_names:
    print(f"Loading & encoding with {name}…")
    m = SentenceTransformer(name)
    emb = m.encode(
        subset['text'].tolist(),
        show_progress_bar=True,
        convert_to_numpy=True
    )
    models[name] = m
    embeddings_dict[name] = emb


Loading & encoding with all-MiniLM-L6-v2…


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [6]:
def search_books(query: str, model_name: str = 'all-MiniLM-L6-v2', top_k: int = 5) -> pd.DataFrame:
    
    q_emb = models[model_name].encode([query], convert_to_numpy=True)
    sims = cosine_similarity(q_emb, embeddings_dict[model_name]).flatten()
    top_idx = sims.argsort()[::-1][:top_k]
    results = subset.loc[top_idx, ['book_name','categories','summaries']].copy()
    results['score'] = sims[top_idx]
    return results


In [7]:
from IPython.display import display

queries = ["space exploration", "business strategy", "historical romance"]
model_names = [
    'all-MiniLM-L6-v2',
    'paraphrase-MiniLM-L6-v2',
    'all-mpnet-base-v2'
]

for q in queries:
    print(f"\n Query: {q!r} ")
    for name in model_names:
        print(f"\nModel: {name} ")
        display(search_books(q, model_name=name, top_k=3))
    



 Query: 'space exploration' 

Model: all-MiniLM-L6-v2 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,book_name,categories,summaries,score
35,Astrophysics for People in a Hurry,science,"talks about the laws of nature, physics, astr...",0.421647
207,Moonwalking With Einstein,science,not only educates you about the history of me...,0.417730
103,Brief Answers To The Big Questions,science,tackles some of the universe’s biggest myster...,0.417257



Model: paraphrase-MiniLM-L6-v2 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,book_name,categories,summaries,score
298,A Tale of Two Cities,politics,tells the stories of two connected families i...,0.466737
308,A Tale of Two Cities,politics,tells the stories of two connected families i...,0.466737
190,Why We Love,science,"delivers a scientific explanation for love, s...",0.369014



Model: all-mpnet-base-v2 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,book_name,categories,summaries,score
298,A Tale of Two Cities,politics,tells the stories of two connected families i...,0.453510
308,A Tale of Two Cities,politics,tells the stories of two connected families i...,0.453510
221,On Writing,biography,details Stephen King’s journey to becoming on...,0.356653
